In [12]:
import numpy as np
import os
from f_meas_gen import *

In [13]:
#Carrega Rede
#Ybus_File = "ieee-6-bus.txt"
Ybus_File = "Rede_2224bus_GB.txt"
Ybus = np.loadtxt(Ybus_File, dtype='i', delimiter=',')
terminal_buses = get_terminal_bus(Ybus)

In [14]:
def count_branches(Ybus):
    n_branches = 0
    
    for line in Ybus:
        for i in line:
            n_branches += i
    n_branches = int(n_branches/2)
    return n_branches

In [15]:
class Bus_system:
    def __init__(self, Ybus):
        self.Ybus=Ybus
        self.n_bus =  np.size(Ybus,1)
        self.n_branches = count_branches(Ybus)
        self.max_meas = self.n_branches*2 + self.n_bus


In [16]:
def get_initial_meas_plan(meas_plan, terminal_buses):
    for bus in terminal_buses:
        for line in range(meas_plan.shape[0]):
            if (meas_plan[line,1] == bus+1):
                meas_plan[line,6] = 1

    return meas_plan

In [17]:
#Parametros da Topologia
power_system = Bus_system(Ybus)


In [18]:
def build_measurement_plan(Power_Sys,max_redun,semente = 5):

        seed(semente)

        Ybus = Power_Sys.Ybus
        max_meas = Power_Sys.max_meas
        n_bus = Power_Sys.n_bus

        meas_plan = build_empty_measurement_plan(Ybus,max_meas)

        n_meas = 0
        observable = False
        redundancy = 0

        possible_meas = [i for i in range(1,max_meas+1)]

        #Alcança Redundância minima
        while( redundancy < max_redun):

            n_meas += add_random_measurement(meas_plan,possible_meas)
            redundancy =  calculate_redundancy(n_meas,n_bus,max_meas)

        #TODO Criar função 'test_observability(Ybus,meas_plan)' para estas 3 funções 
        H = build_jacobian_matrix(Ybus,meas_plan)
        G = build_gain_matrix(H)
        observable = test_observability(G,1.E-10)
        
        #Alcança observabilidade
        while( not observable):
            n_meas += add_random_measurement(meas_plan,possible_meas)
            redundancy =  calculate_redundancy(n_meas,n_bus,max_meas)

            H = build_jacobian_matrix(Ybus,meas_plan)
            G = build_gain_matrix(H)
            observable = test_observability(G,1.E-10)

        meas_plan = remove_desactivated_measurements(meas_plan)

        return meas_plan

In [25]:
def build_measurement_plan_with_initial_allocation(Power_Sys,max_redun,semente = 5):

        seed(semente)

        Ybus = Power_Sys.Ybus
        max_meas = Power_Sys.max_meas
        n_bus = Power_Sys.n_bus

        empty_plan = build_empty_measurement_plan(Ybus, max_meas)
        terminal_buses = get_terminal_bus(Ybus)
        meas_plan = get_initial_meas_plan(empty_plan, terminal_buses)

        n_meas = sum(meas_plan[:,6])
        observable = False
        redundancy = calculate_redundancy(n_meas,n_bus,max_meas)

        possible_meas = [i for i in range(1,max_meas+1)]
        pre_allocated_meas = [i for i in meas_plan[:,0] if meas_plan[i-1,6] == 1]
        available_meas = list(set(possible_meas) - set(pre_allocated_meas))

        # #Alcança Redundância minima
        while( redundancy < max_redun):

            n_meas += add_random_measurement(meas_plan,available_meas)
            redundancy =  calculate_redundancy(n_meas,n_bus,max_meas)

        #TODO Criar função 'test_observability(Ybus,meas_plan)' para estas 3 funções 
        H = build_jacobian_matrix(Ybus,meas_plan)
        G = build_gain_matrix(H)
        observable = test_observability(G,1.E-10)
        
        #Alcança observabilidade
        while( not observable):
            n_meas += add_random_measurement(meas_plan,possible_meas)
            redundancy =  calculate_redundancy(n_meas,n_bus,max_meas)

            H = build_jacobian_matrix(Ybus,meas_plan)
            G = build_gain_matrix(H)
            observable = test_observability(G,1.E-10)

        meas_plan = remove_desactivated_measurements(meas_plan)

        return meas_plan

In [26]:

measurement_plan = build_measurement_plan_with_initial_allocation(power_system,.6)

UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM
UNOBSERVABLE SYSTEM


In [27]:
#Tamanho da Rede
print(np.size(measurement_plan,0))
#Salva Rede
np.savetxt('medplan08-12-22.txt',measurement_plan,fmt='%1d')

5673
